## Q.1 Implement the forward propagation for a two hidden layer network for m-samples, n-features as we discussed in class. Initialize the weights randomly. Use the data from the previous labs like logistic regression. You can choose the number of neurons in the hidden layer and use sigmoid activation function.Report the evaluation metrics for the network.  Also use other non-linear activation functions like ReLU and Tanh. Report the loss using both MSE and Cross Entropy.



In [88]:
import numpy as np
import pandas as pd
    
def softmax(x):
    x_max = np.max(x, axis=0, keepdims=True)
    e_x = np.exp(x - x_max)
    return e_x / np.sum(e_x, axis=0, keepdims=True)
    
# activation function so that we can change it
def activation(x, active):
    if active == "sigmoid":
        return 1 / (1 + np.exp(-x))
    elif active == "relu":
        return np.maximum(0, x)
    elif active == "tanh":
        return np.tanh(x)
    else:
        raise ValueError("Unknown activation function")

In [89]:
# Initialize parameters

def init_parameters(input_size, hidden_size1, hidden_size2, output_size):
    np.random.seed(0)
    parameters = {}
    parameters['W1'] = np.random.randn(hidden_size1, input_size) 
    parameters['b1'] = np.zeros((hidden_size1, 1))
    parameters['W2'] = np.random.randn(hidden_size2, hidden_size1)
    parameters['b2'] = np.zeros((hidden_size2, 1))
    parameters['W3'] = np.random.randn(output_size, hidden_size2)
    parameters['b3'] = np.zeros((output_size, 1))
    
    return parameters

In [90]:
# Data file
import pandas as pd
df = pd.read_csv('Logistic_regression_ls.csv')
print(df)

        x1     x2  label
0    7.395  7.638      1
1    4.987  6.485      1
2    5.358  6.499      1
3    2.036  2.380      0
4    5.956  7.378      1
..     ...    ...    ...
495  0.304  1.608      0
496  6.140  4.261      1
497  6.579  6.231      1
498  2.555  0.446      0
499  2.148  0.852      0

[500 rows x 3 columns]


In [138]:
# Take input and labels from file

X = df[['x1', 'x2']].values
y = df['label'].values.reshape(-1, 1)
X = X.T # Transpose to multiply it with W

print(X.shape)
print(y.shape)

(2, 500)
(500, 1)


In [98]:
def forward_propagation(X, parameters, activation_fn):
    
    # get parameters from dict
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    W3 = parameters['W3']
    b3 = parameters['b3']
    
    # Forward propagation
    Z1 = np.dot(W1, X) + b1
    A1 = activation(Z1, activation_fn)
    Z2 = np.dot(W2, A1) + b2
    A2 = activation(Z2, activation_fn)
    Z3 = np.dot(W3, A2) + b3
    y_pred = sigmoid(Z3)
    
    # updated_dict = {'Z1': Z1, 'A1': A1, 'Z2': Z2, 'A2': A2, 'Z3': Z3, 'A3': A3}
    
    return y_pred.T

In [99]:
input_size = X.shape[0]
hidden_size1 = 10
hidden_size2 = 10
output_size = 1

# initialize parameters
parameters = init_parameters(input_size, hidden_size1, hidden_size2, output_size)
# print(parameters)


In [123]:
# Forward prop for different activations

y_pred1 = forward_propagation(X, parameters, activation_fn="sigmoid")
y_pred2 = forward_propagation(X, parameters, activation_fn="relu")
y_pred3 = forward_propagation(X, parameters, activation_fn="tanh")

# print(y_pred1)
# print(y_pred2)
# print(y_pred3)

In [110]:
# MSE
def mean_squared_error(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)
    
# Cross entropy
def binary_cross_entropy(y_true, y_pred):
    epsilon = 1e-15  # to prevent log(0) cases
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
    return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

In [111]:
# print("Output predictions (A3) shape:\n", y_pred.shape)

MSE1 = mean_squared_error(y, y_pred1)
MSE2 = mean_squared_error(y, y_pred2)
MSE3 = mean_squared_error(y, y_pred3)

print('MSE Using Sigmoid activation function', MSE1)
print('MSE Using Relu activation function', MSE2)
print('MSE Using tanh activation function', MSE3)

MSE Using Sigmoid activation function 0.41988458946454016
MSE Using Relu activation function 0.4999971182583223
MSE Using tanh activation function 0.36545125350493163


In [112]:
# Cross Entropy Loss(CEL)

CEL1 = binary_cross_entropy(y, y_pred1)
CEL2 = binary_cross_entropy(y, y_pred2)
CEL3 = binary_cross_entropy(y, y_pred3)

print('Cross Entropy loss Using Sigmoid activation function', CEL1)
print('Cross Entropy loss Using Relu activation function', CEL2)
print('Cross Entropy loss Using tanh activation function', CEL3)

Cross Entropy loss Using Sigmoid activation function 1.263801246419863
Cross Entropy loss Using Relu activation function 14.895921238585863
Cross Entropy loss Using tanh activation function 1.0287023477610566


In [125]:
def predict_labels(output_probs, threshold=0.5):
    # Convert output probabilities to binary labels
    return (output_probs >= threshold).astype(int)

# Convert output probabilities to binary labels using a threshold = 0.5
y_pred_labels1 = predict_labels(y_pred1)
y_pred_labels2 = predict_labels(y_pred2)
y_pred_labels3 = predict_labels(y_pred3)

# print(y_pred_labels.shape)

In [139]:
def evaluation_metric(y_true, y_pred):
    # accuracy
    accuracy = np.mean(y_true == y_pred)
    # precision
    precision = np.sum((y_true == 1) & (y_pred == 1)) / np.sum(y_pred == 1)
    # recall
    recall = np.sum((y_true == 1) & (y_pred == 1)) / np.sum(y_true == 1)
    # F1-score
    f1_score = 2 * precision * recall / (precision + recall)
    
    return accuracy, precision, recall, f1_score

# Evaluation metric of the model
accuracy1, precision1, recall1, f1_score1 = evaluation_metric(y, y_pred_labels1)
print('Evaluation metric Using Sigmoid activation function:')

print("Accuracy:", accuracy1)
print("Precision:", precision1)
print("Recall:", recall1)
print("F1 Score:", f1_score1)


Evaluation metric Using Sigmoid activation function:
Accuracy: 0.5
Precision: 0.5
Recall: 1.0
F1 Score: 0.6666666666666666


In [140]:
accuracy2, precision2, recall2, f1_score2 = evaluation_metric(y, y_pred_labels2)
print('Evaluation metric Using Relu activation function:')

print("Accuracy:", accuracy2)
print("Precision:", precision2)
print("Recall:", recall2)
print("F1 Score:", f1_score2)

Evaluation metric Using Relu activation function:
Accuracy: 0.5
Precision: 0.5
Recall: 1.0
F1 Score: 0.6666666666666666


In [141]:
accuracy3, precision3, recall3, f1_score3 = evaluation_metric(y, y_pred_labels3)
print('Evaluation metric Using tanh activation function:')

print("Accuracy:", accuracy3)
print("Precision:", precision3)
print("Recall:", recall3)
print("F1 Score:", f1_score3)

Evaluation metric Using tanh activation function:
Accuracy: 0.504
Precision: 0.5020080321285141
Recall: 1.0
F1 Score: 0.6684491978609626
